In [2]:
print("Setting everything up!")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import push_notebook, show, output_notebook
output_notebook()
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, FixedTicker
import bokeh.palettes as pal
from bokeh.layouts import layout, Spacer, gridplot

Setting everything up!


Loading BokehJS ...

In [3]:
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display, clear_output

In [4]:
import numpy as np
from bokeh.charts import Bar, Histogram

/home/sychen23/.local/lib/python3.5/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


In [5]:
from analysis import classify_image, glimpses, read_n, classify_imgs2

['/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py', 'true', 'true', 'true', 'true', 'true', 'true', 'model_runs/regimen/classify_log.csv', 'model_runs/regimen/classifymodel_0.ckpt', 'model_runs/regimen/classifymodel_', 'model_runs/regimen/zzzdraw_data_5000.npy', 'false', 'true', 'false', 'false', 'true']
analysis.py


In [6]:
# import numpy as np
# import scipy.special

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from random import randint

from scipy import linspace
from scipy import pi,sqrt,exp
from scipy.special import erf
import scipy

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show

In [18]:
def get_pdf(sigma, x, mu):
    """Calculate the pdf."""
    
#     pdf = 1/(x* sigma * np.sqrt(2*np.pi)) * np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
    pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    return pdf


def get_cdf(sigma, x, mu, a):
    """Calculate the cdf."""
    
#     cdf = (1 + erf(a*x / sqrt(2))) / 2 
    cdf = (1 + scipy.special.erf((a*x - mu) / np.sqrt(2*sigma**2))) / 2
    return cdf


def get_p(pdf, cdf):
    """Calculate p to create skew curve."""
    
    p = 2 / pdf * cdf / 100000
    return p

    
def curve(sigma, x, mu, plot, label="", color="gray"):
    """Add normal distribution curve to plot."""
    
    pdf = get_pdf(sigma, x, mu)
    min_blobs = 1
    plot.line(x + min_blobs, pdf, line_color=color, line_width=8, alpha=1, legend=label)
    
    
def skew_curve(sigma, x, mu, a, plot):
    """Add a skewed curve to the plot."""
    
    p = get_p(get_pdf(sigma, x, mu), get_cdf(sigma, x, mu, a))

    def f(x): return 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    max_x = scipy.optimize.fmin(lambda x: -f(x), a)

    plot.line(9-x, p / f(max_x), line_color="blue", line_width=8, alpha=0.4)
    
    
def combined_curve(sigma, x, mu, a, plot):
    """Add curve combining skew and normal distribution to plot."""
    
    p = get_p(get_pdf(sigma, x, mu), get_cdf(sigma, x, mu, a))
    
    def f(x): return 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    max_x = scipy.optimize.fmin(lambda x: -f(x), a)

    def f2(x): return (1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2)) + p / f(max_x))
    max_x2 = scipy.optimize.fmin(lambda x: -f(x), a)

    plot.line(x, (((1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2)))
            + p / f(max_x))/ f2(max_x2)), line_color="purple", line_width=2, alpha=1)

In [23]:
def set_figure_colors(p, bg, fg):
    """Set figure p background colors bg and foreground colors fg."""
    
    p.border_fill_color = bg
    p.title.text_color = fg
    p.xaxis.axis_label_text_color = fg
    p.yaxis.axis_label_text_color = fg
    p.xaxis.axis_line_color = fg
    p.yaxis.axis_line_color = fg
    p.xaxis.major_label_text_color = fg
    p.yaxis.major_label_text_color = fg
    p.xaxis.major_tick_line_color = fg
    p.xaxis.minor_tick_line_color = fg
    p.yaxis.major_tick_line_color = fg
    p.yaxis.minor_tick_line_color = fg

In [28]:
clear_output()
b2 = Button(description="Click to Start", icon="arrow", width=400)

dropdown2 = Dropdown(options=['0', '1000', '2000', '3000', '4000', '5000', '10000', '20000', '30000', '40000', '50000',
                             '60000', '70000', '80000', '90000', '100000', '110000', '120000', '130000', '140000', '150000',
                             '160000', '170000', '180000', '190000', '200000', '250000', '300000', '400000', '500000',
                             '600000', '700000', '800000', '900000', '910000', '920000', '1000000', '1100000', '1200000',
                              '1300000', '1400000', '1500000'],
                    value='10000', 
                    description='Iteration:'
)

data = None

def update_curves():
    clear_output()
    global data
    num_imgs = 9000
    print("number of images: %d" % num_imgs)
    imgs_data = classify_imgs2(int(dropdown2.value), True, num_imgs)
    
#     num_blobs = randint(0, 9)
    max_blobs = 9
    min_blobs = 1
    
    curves = list()
    dark = "#111111"
    light = "#DDDDDD"
    p2 = figure(title="Blob Number Classification Probabilities Distributions", y_range=(0, 1), tools="save", background_fill_color=dark)
    set_figure_colors(p2, dark, light)


    for num_blobs in range(min_blobs, max_blobs + 1):
        print("number of blobs: ", num_blobs)
        

        p1 = figure(title="Blob Number Classification Probabilities Distribution for %d Blobs" % num_blobs , y_range=(0, 1), tools="save",
                    background_fill_color=dark)
        set_figure_colors(p1, dark, light)

        m = 0.1
        z_size = max_blobs - min_blobs + 1

        new_hist = np.zeros(z_size)
        choice_hist = np.zeros(z_size)
        value_counts = np.zeros(z_size)
        values_sum = 0
        sqr_sum = 0
        num_imgs_with_num_blobs = 0

        for idx, data in enumerate(imgs_data):

            if data["label"][(num_blobs - min_blobs)] == 1: # data is for an image with num_blobs blobs
                num_imgs_with_num_blobs += 1
                
                max_glimpse = 2
                min_glimpse = 0
                glimpses = 10#max_glimpse - min_glimpse + 1
                
                for i in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:#range(min_glimpse, max_glimpse + 1):
                    # Histogram of softmaxes
                    new_hist += data["classifications"][i][0] / glimpses
                    
                    # Histogram of choices
                    choice = np.argmax(data["classifications"][i][0])
                    choice_list = [0] * z_size
                    choice_list[choice] = 1 / glimpses
                    choice_hist += choice_list
                    
#                 glimpse = 1
        
#                 new_hist += data["classifications"][glimpse][0]
        
#                 choice = np.argmax(data["classifications"][glimpse][0])
#                 choice_list = [0] * z_size
#                 choice_list[choice] = 1
#                 choice_hist += choice_list
                
        print("num_imgs_with_num_blobs: ", num_imgs_with_num_blobs)
        
        new_hist = new_hist / num_imgs_with_num_blobs
        print("new_hist: ", new_hist)
        choice_hist = choice_hist / num_imgs_with_num_blobs

        x = np.linspace(-2, 11.0, 1000)
        source = ColumnDataSource(data=dict(color=["red"] * z_size, top=new_hist, bottom=np.zeros(z_size), left=np.arange(min_blobs, max_blobs + 1) + m - 0.55, right=np.arange(min_blobs + 1, max_blobs + 2) - m - 0.55))
        source2 = ColumnDataSource(data=dict(color=["yellow"] * z_size, top=choice_hist, bottom=np.zeros(z_size), left=np.arange(min_blobs, max_blobs + 1) + m - 0.45, right=np.arange(min_blobs + 1, max_blobs + 2) - m - 0.45))
        source.data["color"][(num_blobs - min_blobs)] = "lime"
        p1.quad('left', 'right', 'top', 'bottom', source=source, color="color", alpha=1)
        p1.quad('left', 'right', 'top', 'bottom', source=source2, color="color", alpha=0.5)


        # FORMAT PLOT ##############################

        p1.xaxis.axis_label = 'Number of Blobs'
        p1.yaxis.axis_label = 'Classification Probability'
        p1.xaxis[0].ticker=FixedTicker(ticks=np.arange(min_blobs, max_blobs + 1))
        
        
        # PLOT CURVES #############################
        
        # Find the mean
        for j in range(z_size):
            values_sum += j * new_hist[j]
        mu = values_sum
        print("mu: ", mu)

        # Find the standard deviation
        for k in range(z_size):
            sqr_sum += new_hist[k] * ((k - mu) ** 2)
        sigma = np.sqrt(sqr_sum)
        
        curves.append((sigma * 2, x, mu, p2, str(num_blobs),
                       "#" + str(randint(2, 9)) + str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))))
        
        curve(sigma, x, mu, p1) # plot gaussian curve
#         a = np.argmax(new_hist)
#         skew_curve(sigma, x, mu, a, p1)
#         combined_curve(sigma, x, mu, a, p1)

        show(gridplot(p1, ncols=2, plot_width=500, plot_height=400, toolbar_location=None))
            
    for stats in curves:
        curve(*stats)
    show(p2)
    

def on_click2(b2, new_image=True):
    """Load new random image after button is clicked."""
    
    b2.description = "Loading..."
    update_curves()
    b2.description = "Next (Random) Image"

b2.on_click(on_click2)


def on_change2(change):
    """Change the iteration number to new dropdown selection."""
    
    if change['type'] == 'change' and change['name'] == 'value':
        on_click(b2, new_image=False)
        

dropdown2.observe(on_change2)
display(HBox([b2, dropdown2]))

number of images: 9000
INFO:tensorflow:Restoring parameters from model_runs/DRAM_test_square/classifymodel_400000.ckpt
number of blobs:  1
num_imgs_with_num_blobs:  1000
new_hist:  [  9.88758520e-01   1.10968592e-02   1.42114933e-04   2.19140818e-06
   3.04986469e-07   8.12809966e-09   3.57514833e-09   2.50747210e-09
   3.22809195e-10]
mu:  0.011388965507


number of blobs:  2
num_imgs_with_num_blobs:  1000
new_hist:  [  3.06039930e-02   8.37561064e-01   1.24782999e-01   6.40141563e-03
   6.20243366e-04   2.88854740e-05   1.28572851e-06   1.06487155e-07
   1.08216338e-08]
mu:  1.10896525655


number of blobs:  3
num_imgs_with_num_blobs:  1000
new_hist:  [  5.03574745e-04   2.52113661e-01   5.48626873e-01   1.43457494e-01
   4.60791809e-02   8.07765042e-03   9.57076524e-04   1.59099003e-04
   2.53948437e-05]
mu:  2.01150417426


number of blobs:  4
num_imgs_with_num_blobs:  1000
new_hist:  [  2.27095782e-07   2.87726729e-02   3.34433070e-01   2.98039020e-01
   2.30117793e-01   7.92885403e-02   2.28966957e-02   5.43919103e-03
   1.01279081e-03]
mu:  3.09222658667


number of blobs:  5
num_imgs_with_num_blobs:  1000
new_hist:  [  3.41435784e-12   2.18507818e-03   1.02092614e-01   2.08028216e-01
   3.28149055e-01   2.06402765e-01   1.04609254e-01   3.82050663e-02
   1.03279519e-02]
mu:  4.15277960212


number of blobs:  6
num_imgs_with_num_blobs:  1000
new_hist:  [  2.78356272e-15   2.31420588e-04   2.23159095e-02   8.84440016e-02
   2.44831366e-01   2.55190403e-01   2.19253121e-01   1.21580024e-01
   4.81537552e-02]
mu:  5.11728165853


number of blobs:  7
num_imgs_with_num_blobs:  1000
new_hist:  [  2.04831311e-19   3.42050853e-06   3.78870813e-03   2.86823664e-02
   1.24132519e-01   2.04880960e-01   2.79289525e-01   2.29476478e-01
   1.29746024e-01]
mu:  5.93460349567


number of blobs:  8
num_imgs_with_num_blobs:  1000
new_hist:  [  1.12781810e-18   1.50119348e-06   6.39377430e-04   7.66143772e-03
   4.81804330e-02   1.19869884e-01   2.58829024e-01   3.10234637e-01
   2.54583707e-01]
mu:  6.57762197513


number of blobs:  9
num_imgs_with_num_blobs:  1000
new_hist:  [  2.86545968e-26   1.06369189e-09   5.21651012e-05   1.35475700e-03
   1.64460876e-02   6.52634704e-02   2.04890981e-01   3.35597585e-01
   3.76394953e-01]
mu:  6.98595890828
